# Group 12 Coursework 

In [1]:
import os
import datetime
import time
import json
import http.client
import requests
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
# This is a class for parsing the html returned from the web service endpoint

    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self.emotion_type = None
        self.emotion_return = False
        
    def handle_starttag(self, tag, attrs):
        if tag=='h3':
            self.emotion_return = True
    
    def handle_data(self, data):
        if self.emotion_return:
            self.emotion_type = data.strip()
        
    def handle_endtag(self, tag):
        if tag=='h3':
            self.emotion_return = False

def deploy_changes(commitmsg):
    '''Run git commands to deploy current changes in project folder to Github and deploy to web server'''
    
    !git add .
    !git commit -m commitmsg
    !git push origin master
    print('Changes were deployed to Github. Please action the CI/CD pipeline running status.')
    
    

### 1. Submit user input for classifying emotion
Below cell will read user input text, and then make a HTTP request using POST method to the webservice endpoint deployed locally (http://127.0.0.1:5000) to classify the emotion of the user input text.

In [2]:
usertext = input("Please input your text:  ")

Please input your text:  hello


In [7]:
data = {'input_text': usertext}
dbipaddr = '127.0.0.1:5000'

response = requests.post('http://' + dbipaddr, data=data)

parser = MyHTMLParser()

if response.status_code==200:       
    parser.feed(response.text)
    if parser.emotion_type is not None:
        print(f'The classified emotion type is: {parser.emotion_type}')
    else:
        print(f'No emotion can be classified!')

The classified emotion type is: neutral


### 2. Apply code changes to CI/CD Pipeline in Github

In [9]:
deploy_changes("updated index.py to return log entry.")

[master 951bc2c] commitmsg
 1 file changed, 5 insertions(+), 2 deletions(-)
Changes were deployed to Github. Please action the CI/CD pipeline running status.


remote: Resolving deltas:   0% (0/2)        
remote: Resolving deltas:  50% (1/2)        
remote: Resolving deltas: 100% (2/2)        
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.        
To https://github.com/wongp1984/comm061nlp2.git
   6eb5638..951bc2c  master -> master


### 3. Get log entry from web server for monitoring

In [38]:
response = requests.get('http://' + dbipaddr + '/getlog')

# parser = MyHTMLParser()

if response.status_code==200:
    lines = response.text.split('</p>')
    for l in lines:    
        print(l[3:])
       
 

input_time,input_text,predcition_time,result
'2023-05-16 23:02:28.931326', 'hello', '2023-05-16 23:02:29.108702', 'neutral'
'2023-05-16 23:32:27.766277','oh my god','2023-05-16 23:32:27.888197','surprise'
'2023-05-16 23:32:33.855147','hey how are you','2023-05-16 23:32:33.970676','confusion_curiosity'
'2023-05-16 23:32:39.957783','life sucks','2023-05-16 23:32:40.065766','anger_annoyance_disgust'
'2023-05-16 23:32:45.938526','life really sucks','2023-05-16 23:32:46.183612','disappointment_embarrassment_grief_remorse_sadness'



### Performance Test

In [ ]:
#performance test script
import requests
from concurrent.futures import ThreadPoolExecutor

import random, string

def makeText(n):
    message="The king is good. "   
    text = [message[i % len(message)] for i in range(n)]
    return ''.join(text)

In [ ]:
print(makeText(30))
print(makeText(60))

In [ ]:
def getEmotion(id,message):
    #print("id:",str(id))
    try:
        url = 'http://127.0.0.1:5000'
        res = requests.get(url)
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        res = requests.post(url=url, data=message, headers=headers)
    except IOError:
        print("Failed to open",host)
    return res.elapsed.total_seconds()

In [ ]:
rrent_num = 10
text_num = 100

runs=[value for value in range(concurrent_num)]
message = makeText(text_num)
message = [message]*concurrent_num

with ThreadPoolExecutor(max_workers=concurrent_num) as executor:
    results = executor.map(getEmotion,runs,message)

In [ ]:
elapsed_times = []
for result in results:
    elapsed_times.append(result)
    #print(result.elapsed.total_seconds())

In [ ]:
print(len(elapsed_times))
print(sum(elapsed_times)/len(elapsed_times))

In [ ]:
concurrent_num = 100
runs=[value for value in range(concurrent_num)]
message = "input_text=The king is good"
message = [message]*concurrent_num

with ThreadPoolExecutor(max_workers=concurrent_num) as executor:
    results = executor.map(getEmotion,runs,message)